In [5]:
#Using pretrained model

import tensorflow as tf
from tensorflow.keras.applications import VGG16
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

img_dim = (224,224, 3)
base_model = VGG16(input_shape=img_dim, weights='imagenet', include_top=True)

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions

image = load_img('/Users/tengyaolong/Desktop/download-2.jpg')
image = img_to_array(image)  #output Numpy-array

image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

image = preprocess_input(image)
yhat = base_model.predict(image)

label = decode_predictions(yhat)
print(label)

[[('n04285008', 'sports_car', 0.9026759), ('n02974003', 'car_wheel', 0.032814305), ('n04037443', 'racer', 0.031918023), ('n02814533', 'beach_wagon', 0.01917579), ('n03100240', 'convertible', 0.007558585)]]


In [ ]:
#what if we want to use this pretrained model to predict different classes other than the ones from ImageNet?
#we replace the last fully connected layers of the pretrained model and train them on new classes (include_top=False)

base_model = VGG16(input_shape=img_dim, weights='imagenet', include_top=False)

#freeze the model
base_model.trainable=False

#create new fully connected layer
prediction_layer = tf.keras.layers.Dense(20, activation='softmax')

#add new fully connected layer to our base model
new_model = tf.keras.Sequential([base_model, prediction_layer])

#finally we train the model, but only the weights of the last layer will be updated

optimizer = tf.keras.optimizers.Adam(0.001)
new_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
new_model.fit(features_train, label_train, epoch=5, validation_split=0.2, verbose=2)